<a href="https://colab.research.google.com/github/andresbadillo/desviaciones/blob/master/Desviaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desviaciones
- Cargar el archivo en pdf con todas las desviaciones "bien escaneadas"

In [ ]:
!sudo apt install tesseract-ocr poppler-utils
!pip install pdf2image pytesseract PyPDF2

import os
import re
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
from PyPDF2 import PdfReader, PdfWriter
from google.colab import files
from IPython.display import HTML

def procesar_pdf():
    # Configurar rutas y carpetas
    if not os.path.exists('temp_images'):
        os.makedirs('temp_images')
    if not os.path.exists('paginas'):
        os.makedirs('paginas')

    # Subir archivo PDF
    uploaded = files.upload()
    pdf_path = list(uploaded.keys())[0]

    # Convertir PDF a imágenes
    images = convert_from_path(pdf_path, dpi=300, output_folder='temp_images', fmt='jpeg')

    # Procesar cada página
    reader = PdfReader(pdf_path)
    nius = {}

    for page_num, image in enumerate(images):
        # 1. Recortar área de NIU (ajustar coordenadas según necesidad)
        width, height = image.size
        area_niu = (width * 0.7, height * 0.05, width * 0.95, height * 0.12)  # Superior derecha
        cropped = image.crop(area_niu)

        # 2. OCR en el área recortada
        text = pytesseract.image_to_string(cropped, config='--psm 6').strip()

        # 3. Buscar número de al menos 3 dígitos
        match = re.search(r'\b\d{3,}\b', text)

        # Si se encuentra, asignar el NIU; si no, usar un nombre genérico
        niu = match.group(0) if match else f"unknown_NIU_{page_num + 1}"

        nius[page_num + 1] = niu

        # Guardar para debug
        cropped.save(f'temp_images/pag_{page_num + 1}_niu_{niu}.jpg')

    # Dividir PDF y renombrar
    for page_num in range(len(reader.pages)):
        writer = PdfWriter()
        writer.add_page(reader.pages[page_num])

        output_name = f"paginas/{nius[page_num + 1]}.pdf"

        with open(output_name, "wb") as out_pdf:
            writer.write(out_pdf)

    # Comprimir y descargar
    !zip -r resultados.zip paginas/ temp_images/
    files.download("resultados.zip")

# Ejecutar el proceso
procesar_pdf()

## Borrar la carpeta `content`
- cambiar el final de la ruta con `paginas` o `temp_images`


In [11]:
import os

# Ruta de la carpeta predeterminada en Google Colab
folder_path = '/content/paginas'

# Eliminar todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

print("\nTodos los archivos cargados han sido eliminados de la carpeta content de Google Colab.")


Todos los archivos cargados han sido eliminados de la carpeta content de Google Colab.
